# Video Augmentation

Este notebook está destinado a la automatización del proceso de Data augmentation aplicado a los vídeos, pues es la fuente de datos para nuestro primer modelo de clasificación. Este método conseguirá agrandar la cantidad de training data que podemos aportar al modelo para evitar tanto el overfitting como para intentar que el modelo pueda obtener buen accuracy sin necesidad de tener la mejor data


## Problemas del Pose Tracking

Este tipo de modelizado de una persona en 2D en base a un video aportado al modelo es el método seleccionado para intentar estudiar la estructura del cuerpo de la persona del video, pero este modelizado puede fallar por diversas razones: Los videos que la gente puede pasar pueden tener ángulos que no den pie a detectar bien el cuerpo, teniendo objetos de por medio, con condiciones de luz escasas que den problemas para distinguir a la persona, etc. 
Es por ello que se intentará que el modelo funcione mejor gracias a tener muchos datos "malos" que sepa que tiene que reconocer.

# Imports

In [1]:
import os
import cv2
import random
import numpy as np
import vidaug.augmentors as va
from PIL import Image, ImageSequence
workpath = 'C:/Users/Legion/TFM/Tareas'
os.chdir(workpath)

# Function definitions

In [2]:
def video_loader(video_path):
    # Abrir el video con OpenCV
    vidcap = cv2.VideoCapture(video_path)
    frames = []
    success, image = vidcap.read()
    
    while success:
        # Convertir cada frame de BGR (formato OpenCV) a RGB (formato Pillow)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Convertir el frame a un objeto PIL.Image.Image
        pil_image = Image.fromarray(image_rgb)
        frames.append(pil_image)
        # Leer el siguiente frame
        success, image = vidcap.read()
    
    vidcap.release()
    return frames


def frames_to_video(frames, output_path, fps=30):
    if not frames:
        print("No hay frames para escribir en el video.")
        return

    # Obtener el tamaño de los frames
    frame_size = frames[0].size  # (width, height)
    frame_width, frame_height = frame_size

    # Crear el VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codificación para MP4
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    for i, frame in enumerate(frames):
        # Convertir el frame de PIL.Image a un array de NumPy
        frame_np = cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR)
        # Escribir el frame en el video
        out.write(frame_np)

    # Liberar el VideoWriter
    out.release()
    print(f"Video guardado en {output_path}")

def frames_to_gif(frames, output_path, duration=100):
    if not frames:
        print("No hay frames para escribir en el GIF.")
        return

    # Convertir los frames a objetos PIL.Image
    pil_images = [frame.convert("RGB") for frame in frames]

    # Guardar como GIF
    pil_images[0].save(
        output_path,
        save_all=True,
        append_images=pil_images[1:],
        duration=duration,
        loop=0
    )
    print(f"GIF guardado en {output_path}")

def create_new_video(video):
    prob100 = lambda aug: va.Sometimes(1, aug) # Used to apply augmentor with 100% probability
    prob80 = lambda aug: va.Sometimes(0.8, aug) # Used to apply augmentor with 80% probability
    prob60 = lambda aug: va.Sometimes(0.6, aug) # Used to apply augmentor with 60% probability
    prob50 = lambda aug: va.Sometimes(0.5, aug) # Used to apply augmentor with 50% probability
    prob30 = lambda aug: va.Sometimes(0.3, aug) # Used to apply augmentor with 30% probability
    seq = va.Sequential([ 
        prob100(va.HorizontalFlip()), # horizontally flip the video with 100% probability
        prob30(va.Pepper()), # Pepper Noise added 30% probability
        prob30(va.Salt()), # Salt Noise added 30% probability
        prob80(va.RandomRotate(random.randint(15, 30))), # Rotation between 15 and 30 degrees, 30% probability
    ])
    return seq(video)

def pipeline_augmentation(input_video_path, output_dir_path):
    video_name = input_video_path.split('/')
    name = video_name[-1].split('.')
    input_frames = video_loader(input_video_path)
    for num in range(4):
        new_video = create_new_video(input_frames)
        frames_to_video(new_video, output_dir_path+'/'+name[0]+'_'+str(num)+'.mp4')
    print('Videos augmented succesfully')

In [3]:
videos = os.listdir(workpath+'/VideosTR/VideosAug')
abs_paths_videos = [workpath+'/VideosTR/VideosAug/'+x for x in videos if '.' in x]
out_path = workpath+'/VideosTR/VideosAug'

In [ ]:
for video in abs_paths_videos:
    pipeline_augmentation(video, out_path)